# Extracting lists to Disavow links on Gooogle Search Console

My brother's website ([LucidGen.com](https://lucidgen.com)) is getting many backlinks of unknown origin. Someone is attacking his website by giving him many bad backlinks. These things can badly affect his website ranking on Google Search. So he needs to find bad backlinks in all of the backlinks to tell with Google that is not his and require Google to exclude it from counting backlinks for his website. I'll help him.

**How I do it:**
I'll export data from Ahref (backlinks), MOZ (Spam Score), Search Console (Top linking sites) and combine them into one dataset.
I also gather some lists for conditions such as Domain White List, Posts Sitemap, Bad Keywords for Domain, and Bad Keywords for Path.

I'll extract Manual Check by use conditions:
- Domain and path not in bad keywords, path not contain his post's path
- Domain Authority (DA) > 15 and Spam Score (SS) < DA*20% < 10%
- Domain Rating (DR) > 30 and Domain Traffic (DT) > 3000

All the remaining domains I will add to the Disavow list.

**Summary of results:**
I get three final lists:
- **Disavow list:** I'll send it to Google Search Console.
- **DMCA Report list:** I'll report to Google Copyright Removal because they infringe his content.
- **Manual Check List:** I need to check because they don't match any conditions. I can use Ahref Quick Batch Analysis to find their DA and add bad backlinks to Disavow list.


## Exploring datasets & lists

### Exploring datasets

#### Ahref dataset
I'll export it from Ahref > Site Explorer > Backlinks.

In [1]:
import pandas as pd
from csv import reader
import tldextract # For extracting TLD domain (Root domain)
from urllib.parse import urlparse # For extracting path in URL

In [2]:
# Explore Ahref dataset
df_ahref = pd.read_csv('Input/Dataset_Ahref.csv')
print(df_ahref.shape)
df_ahref.head()

(3396, 30)


,Referring page title,Referring page URL,Language,Platform,Referring page HTTP code,Domain rating,Domain traffic,Referring domains,Linked domains,External links,...,Sponsored,Rendered,Raw,Lost status,Drop reason,Discovered status,First seen,Last seen,Lost,Links in group
0,Thach Pham – Chuyên trang chia sẻ các kiến thứ...,https://thachpham.com/,NaN,wordpress,200,66.0,20380,699,10,11,...,False,False,True,NaN,NaN,pagefound,2021-10-11 09:11:28,2022-03-19 10:05:34,NaN,1035
1,"Cách kiểm tra tên máy, mã máy, cấu hình laptop...",https://huuthuan.net/kiem-tra-ten-may-ma-may-l...,vi,wordpress,200,31.0,16996,1,21,28,...,False,False,True,NaN,NaN,linkrestored,2020-06-25 10:34:52,2022-03-16 23:32:53,NaN,2676
2,Top công cụ quét số điện thoại trên facebook t...,https://backlink123.com/top-cong-cu-quet-so-di...,NaN,wordpress,200,6.0,15163,0,6,9,...,False,False,True,NaN,NaN,pagefound,2021-02-26 08:02:35,2022-03-15 21:48:26,NaN,5
3,"Cách viết chữ in đậm, in nghiêng trên bài viết...",https://kinhdoanhthantoc.com/cach-viet-chu-in-...,"vi, en","ecommerce, wordpress",200,24.0,1233,1,10,11,...,False,False,True,NaN,NaN,linkrestored,2020-02-25 07:22:59,2022-02-13 16:44:24,NaN,1
4,Tạo chữ đậm chữ nghiêng để copy vào Facebook/ ...,https://dichthuatphuongdong.com/tienich/chu-da...,vi,NaN,200,31.0,193075,0,4,5,...,False,False,True,removedfromhtml,NaN,pagefound,2021-04-01 01:40:52,2021-09-14 06:14:28,2022-03-12 13:11:35,1


I'll use `Referring page URL`, `Domain Rating` and `Domain traffic` from Ahref dataset.

#### Google Seach Console dataset
I'll export it from Search Console > Links > Top linking sites.

In [3]:
# Explore Console dataset
df_console = pd.read_csv('Input/Dataset_Console.csv')
df_console.head()

,Site,Linking pages,Target pages
0,huuthuan.net,5721,1
1,thachpham.com,2864,1
2,medium.com,453,92
3,microsoft.com,417,3
4,angkoo.com,204,32


In [4]:
df_console.shape

(371, 3)

I'll use only `Site` from Console dataset.

#### MOZ dataset
I'll export it from Moz Pro > Link Research > Spam Score.
Because the MOZ dataset has many redundant rows, I have to use another way to view, and I will delete these rows later.

In [5]:
#Explore Moz dataset
open_file = open('Input/Dataset_Moz.csv')
read_file = reader(open_file)
df_moz = list(read_file)

for row in df_moz[0:10]:
    print(row)

['\ufeff------------------------------------------------------------------------']
['Inbound Links by Spam Score for', 'lucidgen.com']
['Scope', 'pld']
['Sorted by', 'spam_score']
['------------------------------------------------------------------------']
[]
['URL', 'Title', 'Anchor Text', 'Spam Score', 'DA', 'Date First Seen', 'Date Crawled']
['www.10k.pw/domain-list-511', '', 'lucidgen.com', '96%', '2', '2021-07-27', '2022-02-02']
['www.100k.pw/domain-list-511', '', 'lucidgen.com', '96%', '2', '2021-07-25', '2022-02-04']
['one.net.in/domain-list-159', 'Alexa top domain list ||  page  159', 'lucidgen.com', '88%', '22', '2022-02-20', '2022-02-20']


In [6]:
df_moz = pd.DataFrame(df_moz[7:],columns=df_moz[6])
df_moz.head()

,URL,Title,Anchor Text,Spam Score,DA,Date First Seen,Date Crawled
0,www.10k.pw/domain-list-511,,lucidgen.com,96%,2,2021-07-27,2022-02-02
1,www.100k.pw/domain-list-511,,lucidgen.com,96%,2,2021-07-25,2022-02-04
2,one.net.in/domain-list-159,Alexa top domain list || page 159,lucidgen.com,88%,22,2022-02-20,2022-02-20
3,s108313.gridserver.com/4fdt8l/how-to-find-my-m...,how to find my messenger code 2021,how to add facebook messenger to website 2022 ...,86%,11,2022-02-08,2022-02-08
4,www.weblists.pw/domain-list-511,,lucidgen.com,86%,3,2021-07-29,2021-12-15


In [7]:
df_moz.shape

(8599, 7)

I'll use `URL`, `Spam Score`, and `DA` from MOZ dataset.

### Exploring the list of uses for the condition

I'll prepare some simple lists with one column and no header:
- **Domain White List:** I use it to exclude the domains from conditions.
- **Posts Sitemap:** I export it from the website sitemap.
- **Bad Keywords for Domain:** For example, top, free, list, domain, .pw, etc.
- **Bad keywords for Path:** For example, list, top, web-site-no, etc.

In [8]:
# Explore the list of uses for the condition.
list_whitelist = pd.read_csv('Input/List_DomainWhitelist.csv',header=None)[0].to_list()
list_sitemap = pd.read_csv('Input/List_PostsSitemap.csv',header=None)[0].to_list()
list_key_domain = pd.read_csv('Input/List_BadKeyDomain.csv',header=None)[0].to_list()
list_key_path = pd.read_csv('Input/List_BadKeyPath.csv',header=None)[0].to_list()

In [9]:
list_whitelist[:5]

['about.me',
 'analyticsmania.com',
 'behance.net',
 'business.site',
 'cellphones.com.vn']

In [10]:
list_sitemap_path = pd.DataFrame(list_sitemap)[0].apply(lambda x: urlparse(x).path.replace('en/','').replace('/','')).to_list()
list_sitemap_path[:5]

['huong-dan-cai-office-mien-phi-tren-macbook',
 'tat-gatekeeper-va-sip-tren-mac',
 'chia-se-va-huong-dan-wordfence-security-premium',
 'viet-chu-in-dam-nghieng-facebook',
 'so-sanh-nen-mua-macbook-air-hay-pro-m1-max']

In [11]:
list_key_domain[:5]

['-', '.biz', '.in', '.info.pl', '.pl']

In [12]:
list_key_path[:5]

['list', 'top', 'web-site-no', 'domain-list', '60000-yen']

## Extracting the required data for datasets

I need to combine all of the datasets into one with columns: Domain, URL, Path, Domain Rating (DR), Domain Authority (DA), Spam Score (SS). So I need to extract some required columns from the datasets.
- **Ahref:** Domain, URL, Path, DR, DT.
- **MOZ:** Domain, URL, Path, DA, SS.
- **Console:** Domain.

And to use for extract conditions, I also extract paths from the posts sitemap.

In [13]:
# Add domain and path columns
df_ahref['Domain'] = df_ahref['Referring page URL'].apply(lambda x : tldextract.extract(x).registered_domain)
df_ahref['Path'] = df_ahref['Referring page URL'].apply(lambda x : urlparse(x).path)

# Remove HTTP/HTTPS to combine datasets later
df_ahref['URL'] = df_ahref['Referring page URL'].apply(lambda x : urlparse(x).netloc + urlparse(x).path)

In [14]:
# Extract the required data for Ahref dataset
df_ahref_extracted = df_ahref[['Domain', 'URL', 'Path', 'Domain rating', 'Domain traffic']]

# Remove domains are null (tldextract will convert ip address domain to null)
df_ahref_extracted = df_ahref_extracted[df_ahref_extracted.Domain != '']
df_ahref_extracted.columns=['Domain', 'URL', 'Path', 'DR', 'DT']
df_ahref_extracted.head()

,Domain,URL,Path,DR,DT
0,thachpham.com,thachpham.com/,/,66.0,20380
1,huuthuan.net,huuthuan.net/kiem-tra-ten-may-ma-may-laptop.html,/kiem-tra-ten-may-ma-may-laptop.html,31.0,16996
2,backlink123.com,backlink123.com/top-cong-cu-quet-so-dien-thoai...,/top-cong-cu-quet-so-dien-thoai-tren-facebook-...,6.0,15163
3,kinhdoanhthantoc.com,kinhdoanhthantoc.com/cach-viet-chu-in-dam-in-n...,/cach-viet-chu-in-dam-in-nghieng-tren-bai-viet...,24.0,1233
4,dichthuatphuongdong.com,dichthuatphuongdong.com/tienich/chu-dam-chu-ng...,/tienich/chu-dam-chu-nghieng.html,31.0,193075


In [15]:
# Add domain and path columns
df_moz['Domain'] = df_moz['URL'].apply(lambda x : tldextract.extract(x).registered_domain)

# urlparse requires HTTP/HTTPS for URL, but MOZ doesn't have
df_moz['Path'] = df_moz['URL'].apply(lambda x : urlparse('https://' + x).path)

# Convert SS to number
df_moz['SS'] = df_moz['Spam Score'].apply(lambda x : x.replace('%','').replace('--','0')).astype(int)

In [16]:
# Extract the required data for Moz dataset
df_moz_extracted = df_moz[['Domain', 'URL', 'Path', 'DA', 'SS']]

# Remove domains are null (tldextract will convert ip address domain to null)
df_moz_extracted = df_moz_extracted[df_moz_extracted.Domain != '']
df_moz_extracted.head()

,Domain,URL,Path,DA,SS
0,10k.pw,www.10k.pw/domain-list-511,/domain-list-511,2,96
1,100k.pw,www.100k.pw/domain-list-511,/domain-list-511,2,96
2,one.net.in,one.net.in/domain-list-159,/domain-list-159,22,88
3,gridserver.com,s108313.gridserver.com/4fdt8l/how-to-find-my-m...,/4fdt8l/how-to-find-my-messenger-code-2021.html,11,86
4,weblists.pw,www.weblists.pw/domain-list-511,/domain-list-511,3,86


In [17]:
# Extract the domains for Console dataset
df_console_domain = pd.DataFrame({'Domain' : df_console.Site})
df_console_domain.head()

,Domain
0,huuthuan.net
1,thachpham.com
2,medium.com
3,microsoft.com
4,angkoo.com


## Merging all datasets into one

### Step 1: Merging distinct domain, URL, Path
Ahref and MOZ datasets have the domain, URL, and Path. But Console dataset does not. So I'll merge Ahref and MOZ first and then Console.

In [18]:
df_domain_url_path = pd.concat([df_ahref_extracted[['Domain', 'URL', 'Path']], df_moz_extracted[['Domain', 'URL', 'Path']]])
df_domain_url_path = pd.concat([df_domain_url_path, df_console_domain]).reset_index(drop=True).drop_duplicates()
df_domain_url_path.head()

,Domain,URL,Path
0,thachpham.com,thachpham.com/,/
1,huuthuan.net,huuthuan.net/kiem-tra-ten-may-ma-may-laptop.html,/kiem-tra-ten-may-ma-may-laptop.html
2,backlink123.com,backlink123.com/top-cong-cu-quet-so-dien-thoai...,/top-cong-cu-quet-so-dien-thoai-tren-facebook-...
3,kinhdoanhthantoc.com,kinhdoanhthantoc.com/cach-viet-chu-in-dam-in-n...,/cach-viet-chu-in-dam-in-nghieng-tren-bai-viet...
4,dichthuatphuongdong.com,dichthuatphuongdong.com/tienich/chu-dam-chu-ng...,/tienich/chu-dam-chu-nghieng.html


In [19]:
df_domain_url_path.shape

(9545, 3)

### Step 2: Merging all data into one dataset
Now, I can merge all data into one dataset with columns: Domain | URL | Path | DR | DT | DA | SS | Disavow.

In [20]:
# Merge all data into one dataset
df_full = pd.merge(df_domain_url_path, df_ahref_extracted[['Domain', 'DR', 'DT']].drop_duplicates(), on='Domain', how='left')
df_full = pd.merge(df_full, df_moz_extracted[['Domain', 'DA', 'SS']].drop_duplicates(), on='Domain', how='left')
df_full

,Domain,URL,Path,DR,DT,DA,SS
0,thachpham.com,thachpham.com/,/,66.0,20380.0,31,1.0
1,huuthuan.net,huuthuan.net/kiem-tra-ten-may-ma-may-laptop.html,/kiem-tra-ten-may-ma-may-laptop.html,31.0,16996.0,21,1.0
2,backlink123.com,backlink123.com/top-cong-cu-quet-so-dien-thoai...,/top-cong-cu-quet-so-dien-thoai-tren-facebook-...,6.0,15163.0,29,25.0
3,kinhdoanhthantoc.com,kinhdoanhthantoc.com/cach-viet-chu-in-dam-in-n...,/cach-viet-chu-in-dam-in-nghieng-tren-bai-viet...,24.0,1233.0,NaN,NaN
4,dichthuatphuongdong.com,dichthuatphuongdong.com/tienich/chu-dam-chu-ng...,/tienich/chu-dam-chu-nghieng.html,31.0,193075.0,NaN,NaN
...,...,...,...,...,...,...,...
11483,52.32.203.101,NaN,NaN,NaN,NaN,NaN,NaN
11484,bibobaby.vn,NaN,NaN,21.0,0.0,NaN,NaN
11485,akinakids.vn,NaN,NaN,NaN,NaN,NaN,NaN
11486,loginsnew.com,NaN,NaN,0.1,59500.0,8,30.0


In [21]:
df_full.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11488 entries, 0 to 11487
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Domain  11488 non-null  object 
 1   URL     11077 non-null  object 
 2   Path    11077 non-null  object 
 3   DR      10039 non-null  float64
 4   DT      10039 non-null  float64
 5   DA      10103 non-null  object 
 6   SS      10103 non-null  float64
dtypes: float64(3), object(4)
memory usage: 718.0+ KB


In [22]:
df_full.DA = df_full.DA.astype(float)

I feel satisfied with this complete dataset. Now I can use it with many conditions to extract to Disavow list.

## Extracting bad domains to Disavow list and DMCA report list

**Conditions for manual check domains:**
- Domain and path not in bad keywords, path not contain his post's path
- Domain Authority (DA) > 15 and Spam Score (SS) < DA*20% < 10%
- Domain Rating (DR) > 30 and Domain Traffic (DT) > 3000

**Conditions for defining a URL infringing content:**
- The path matches his post's path.

### Step 1: Creating some functions for conditions
- is_bad_domain: The domain contains a bad keyword.
- is_bad_path: The path contains a bad key keyword.
- is_copied_path: The path matches his post's path.

In [23]:
# Check if the domain contains a bad keyword
def is_bad_domain(domain):
    for keyword in list_key_domain:
        if '.' in keyword: # Checking suffix exactly ('.in' keyword will not match '.info' suffix)
            if tldextract.extract(domain).suffix in keyword:
                return True
        elif keyword in domain:
            return True
    return False

# Check if the path contains a bad keyword
def is_bad_path(path):
    for keyword in list_key_path:
        if keyword in str(path):
            return True
    return False

# Check if the path matches his post's path
def is_copied_path(path):
    for post in list_sitemap_path:
        if post in str(path):
            return True
    return False

print('Bad domain test:',is_bad_domain('abc.pw'))
print('Bad path test:',is_bad_path('domain-list.html'))
print('Matches path test:',is_copied_path('/digital-marketing/chuyen-doi-tawk-to.html'))

Bad domain test: True
Bad path test: True
Matches path test: True


### Step 2: Extracting bad domains to Disavow list, DMCA report list, and Manual Check list

In [24]:
# Extract bad domains to Disavow list, DMCA report list, and Manual Check list
disavow = []
dmca_report = []
manual_check = []

for row in list(df_full.values):
    domain = row[0]
    path = row[2]
    dr = row[3]
    dt = row[4]
    da = row[5]
    ss = row[6]
    dis = ['domain:'+domain, dr, dt, da, ss] #Removing URL and Path for Disavow list

    # Check if domain in white list
    if domain in list_whitelist:
        pass

    # Check if path matches his post's path
    elif is_copied_path(path):
        disavow.append(dis)
        dmca_report.append(row)

    # Check if domain and path contain a bad keyword
    elif not is_bad_domain(domain) and not is_bad_path(path):
        if dr != 'None' and dr > 30 and dt != 'None' and dt > 3000:
            manual_check.append(row)
        elif da != 'None' and da > 15 and ss != 'None' and ss < da*0.2 and ss < 0.1:
            manual_check.append(row)
        else:
            disavow.append(dis)
    else:
        disavow.append(dis)

In [25]:
header_disavow = ['Disavow domain','DR','DT','DA','SS']
header_full = ['Domain','URL','Path','DR','DT','DA','SS']

In [26]:
df_disavow = pd.DataFrame(disavow, columns=header_disavow).drop_duplicates()
print('Disavow list:', df_disavow.shape[0])
df_disavow.head()

Disavow list: 1259


,Disavow domain,DR,DT,DA,SS
0,domain:backlink123.com,6.0,15163.0,29.0,25.0
1,domain:kinhdoanhthantoc.com,24.0,1233.0,NaN,NaN
2,domain:thanhbinh.me,0.3,170.0,NaN,NaN
3,domain:affviet.com,3.4,77.0,6.0,6.0
4,domain:estore-edu.com,24.0,128.0,11.0,6.0


Now I'll send the Disavow Domain column to Google Search Console. It is distinct (one line per domain).

In [27]:
df_dmca_report = pd.DataFrame(dmca_report,columns=header_full).drop_duplicates()
print('DMCA report list:', df_dmca_report.shape[0])
df_dmca_report.head()

DMCA report list: 954


,Domain,URL,Path,DR,DT,DA,SS
0,top1vietnam.vn,top1leed.top1vietnam.vn/cach-bao-cao-vi-pham-b...,/cach-bao-cao-vi-pham-ban-quyen-google-dmca/,43.0,1005.0,11.0,18.0
1,goodkids.vn,goodkids.vn/cach-go-ung-dung-tren-mac-va-don-d...,/cach-go-ung-dung-tren-mac-va-don-dep-sach-se/,15.0,0.0,NaN,NaN
2,boxplay.vn,www.boxplay.vn/cach-bao-cao-vi-pham-ban-quyen-...,/cach-bao-cao-vi-pham-ban-quyen-google-dmca/,20.0,0.0,NaN,NaN
3,kidsplus.vn,www.kidsplus.vn/cach-tai-final-cut-pro-mien-ph...,/cach-tai-final-cut-pro-mien-phi-cho-mac/,17.0,2.0,NaN,NaN
4,jinjean.com,www.jinjean.com/cong-cu-xoa-dau-tieng-viet-onl...,/cong-cu-xoa-dau-tieng-viet-online-sieu-gon/,19.0,0.0,NaN,NaN


I need to check if these URLs are infringing on his content. If true, I'll send it to Google Copyright Removal.

In [28]:
df_manual_check = pd.DataFrame(manual_check, columns=header_full).drop_duplicates()
print('Manual check list:',df_manual_check.shape[0])
df_manual_check

Manual check list: 276


,Domain,URL,Path,DR,DT,DA,SS
0,sitelike.org,www.sitelike.org/similar/quetsodienthoai.com/,/similar/quetsodienthoai.com/,64.0,14344949.0,NaN,NaN
1,inthienha.com,inthienha.com/viet-chu-in-dam-nghieng-gach-nga...,/viet-chu-in-dam-nghieng-gach-ngang.htm,39.0,31777.0,NaN,NaN
2,updownsite.com,www.updownsite.com/site/lucidgen.com,/site/lucidgen.com,39.0,8332.0,NaN,NaN
3,updownsite.com,www.updownsite.com/search/zalo-pc,/search/zalo-pc,39.0,8332.0,NaN,NaN
4,bingparachute.com,s.bingparachute.com/search,/search,2.6,95.0,21.0,0.0
...,...,...,...,...,...,...,...
271,similars.net,NaN,NaN,42.0,39578.0,NaN,NaN
272,technofizi.net,NaN,NaN,48.0,437588.0,45.0,1.0
273,inthienha.com,NaN,NaN,39.0,31777.0,NaN,NaN
274,advancedsitestats.com,NaN,NaN,32.0,45819.0,27.0,1.0


I'll check it manually with Ahref Quick Batch Analysis to find their DA and add bad backlinks to Disavow list later.

## Saving three final lists to Excel files

In [29]:
#Save three final lists to Excel files
df_disavow.to_excel('Output/Disavow.xlsx',index=None,header=header_disavow)
df_manual_check.to_excel('Output/ManualCheck.xlsx',index=None,header=header_full)
df_dmca_report.to_excel('Output/DmcaReport.xlsx',index=None,header=header_full)

## Conclusion
I have gotten Disavow list, DMCA Report list, and Manual Check list. These lists can help him prevent bad backlinks attacks. He needs to do this work once a week to protect his website rank on Google Search.